In [1]:
import json
import pandas
import utils
import itertools
from itertools import combinations_with_replacement
import copy

# Get the character_stats, weapon stats and artifact stats that you want

In [2]:
character = "characters/soldier11.json"
weapon = "weapons/starlight_engine_r2.json"
disk = "disks_setup/disk_options.json"

# excel_save_name = "results/ellen_sig.xlsx"
recommendations_path = "results/recommendations/soldier11_recommendation.txt"


substat_points = 34

damage_combo = ["ultimate"]
available_disk_sets_options = [
    # "Chaotic_Metal", # Ether
    # "Fanged_Metal", # Physical
    # "Freedom_Blues",
    "Hormone_Punk",
    "Inferno_Metal", # Fire
    # "Polar_Metal", # Ice
    "Puffer_Electro",
    # "Shockstar_Disco", # Impact
    # "Soul_Rock", # DEF
    # "Swing_Jazz",
    # "Thunder_Metal", # Electric
    "Woodpecker_Electro"
]

# Real calculations begin

In [3]:
# load character, weapon, disk, and constants
character_stats, base_motion_values, weapon_stats, other_stats = utils.get_stats(character, weapon, disk)
disk_stats = utils.evaluate_disk_stat(disk_options = other_stats)

print(disk_stats)

total_stats, target_stats = utils.get_total_stats(
    character_stats,
    weapon_stats,
    disk_stats,
)
final_damage = utils.combo_to_damage(
    damage_combo,
    base_motion_values,
    total_stats,
    target_stats
)

print(f"character_stats: {character_stats}")
print(f"weapon_stats: {weapon_stats}")
print(f"other_stats: {other_stats}")
print(f"disk_stats: {disk_stats}")
print(f'total_stats: {total_stats}')
print(final_damage)

{'Flat_HP': 2200, 'Flat_ATK': 316, 'Flat_DEF': 184, 'CRIT_DMG': 1.1520000000000001, 'Elemental_Damage': 0.3, 'ATK_%': 0.5700000000000001, 'CRIT_Rate': 0.264}
character_stats: {'Base_HP': 7672.3, 'Base_DEF': 613.3, 'Base_ATK': 888.57, 'CRIT_Rate': 0.194, 'CRIT_DMG': 0.5, 'PEN_Ratio': 0, 'Impact': 93, 'Anomaly_Mastery': 94, 'Anomaly_Proficiency': 93, 'Energy_Regen': 1.2, 'Element': 'Fire', 'DMG%_Multiplier': 0.325}
weapon_stats: {'Base_HP': 0, 'Base_DEF': 0, 'Base_ATK': 594, 'CRIT_Rate': 0, 'CRIT_DMG': 0, 'PEN_Ratio': 0, 'Impact': 0, 'Anomaly_Mastery': 0, 'Anomaly Proficiency': 0, 'Energy_Regen': 0, 'HP_%': 0, 'DEF_%': 0, 'ATK_%': 0.25, 'Daze_%': 0, 'DMG%_Multiplier': 0, 'Final_HP_%': 0, 'Final_DEF_%': 0, 'Final_ATK_%': 0.138}
other_stats: {'main_3': {'Flat_HP': 2200, 'Flat_ATK': 316, 'Flat_DEF': 184, 'CRIT_DMG': 1.1520000000000001, 'Elemental_Damage': 0.3, 'ATK_%': 0.5700000000000001, 'CRIT_Rate': 0.264}, 'disk_4': {'ATK_%': 0.3, 'HP_%': 0.3, 'DEF_%': 0.48, 'Anomaly Proficiency': 92, 'C

In [4]:
character_stats, base_motion_values, weapon_stats, other_stats = utils.get_stats(character, weapon, disk)

In [5]:
def disk_possibilities(
    character_stats,
    base_motion_values,
    weapon_stats,
    other_stats,
    substat_points = 2,
    max_per_stat = 4,
    sets = [
        "Chaotic_Metal", # Ether
        "Fanged_Metal", # Physical
        "Freedom_Blues",
        "Hormone_Punk",
        "Inferno_Metal", # Fire
        "Polar_Metal", # Ice
        "Puffer_Electro",
        "Shockstar_Disco", # Impact
        "Soul_Rock", # DEF
        "Swing_Jazz",
        "Thunder_Metal", # Electric
        "Woodpecker_Electro"
    ]
):
    best_damage = 0
    best_allocation = None
    # Count occurrences of each substat in the allocation
    valid_substat_combinations = utils.generate_valid_combinations(
        substat_points,
        max_per_stat,
    )
    disk_4_combo, disk_5_combo, disk_6_combo = utils.generate_main_disk_options()

    disk_set_options = utils.generate_valid_disk_sets(sets) # make sure to flip the 2 piece and 4 piece effects
    for disk_4_allocation in disk_4_combo:
        for disk_5_allocation in disk_5_combo:
            for disk_6_allocation in disk_6_combo:
                main_stat_dict = {
                    "disk_4": disk_4_allocation,
                    "disk_5": disk_5_allocation,
                    "disk_6": disk_6_allocation,
                }
                for disk_set_option in disk_set_options:
                    for substat_allocations in valid_substat_combinations:
                        # make a copy of every dictionary in case we directly modify the dictionary
                        other_stats_copy = copy.deepcopy(other_stats)
                        
                        # make a copy of every dictionary in case we directly modify the dictionary
                        disk_stats = utils.evaluate_disk_stat(
                            disk_options = other_stats_copy,
                            main_stat_dict = main_stat_dict,
                            sub_stat_dict = substat_allocations
                        )
                        disk_stats = utils.add_set_options_to_disk(
                            disk_stats,
                            disk_set_option,
                        )
                        total_stats, target_stats = utils.get_total_stats(
                            character_stats,
                            weapon_stats,
                            disk_stats,
                        )
                        final_damage = utils.combo_to_damage(
                            damage_combo,
                            base_motion_values,
                            total_stats,
                            target_stats
                        )
                        if final_damage > best_damage:
                            best_damage = final_damage
                            best_main_stats = main_stat_dict
                            best_allocation = substat_allocations
                            best_total_stats = total_stats
                            best_disk_set_option = disk_set_option
                            print("New Best Damage: ", best_damage)
                            print(f"best_main_stats: {best_main_stats}")
                            print(f"best_allocation: {best_allocation}")
                            print(f"best_total_stats: {best_total_stats}")
                            print(f"best_disk_set_option: {best_disk_set_option}")
                            print("\n\n")

    return [best_damage, best_main_stats, best_allocation, best_total_stats, best_disk_set_option]


outcomes = disk_possibilities(
    character_stats,
    base_motion_values,
    weapon_stats,
    other_stats,
    substat_points = 34,
    max_per_stat=24,
    sets = available_disk_sets_options
    # sets = [
    #     # "Chaotic_Metal", # Ether
    #     # "Fanged_Metal", # Physical
    #     # "Freedom_Blues",
    #     # "Hormone_Punk",
    #     # "Inferno_Metal", # Fire
    #     "Polar_Metal", # Ice
    #     "Puffer_Electro",
    #     # "Shockstar_Disco", # Impact
    #     # "Soul_Rock", # DEF
    #     # "Swing_Jazz",
    #     # "Thunder_Metal", # Electric
    #     # "Woodpecker_Electro"
    # ]
)

print(f"Overall Best Damage: {outcomes[0]}") # Float
print(f"best_main_stats: {outcomes[1]}") # Dict
print(f"best_allocation: {outcomes[2]}") # Dict
print(f"best_total_stats: {outcomes[3]}") # Dict
print(f"best_disk_set_option: {outcomes[4]}") # Tuple
print("\n\n")

# Save to notepad
with open(recommendations_path, "a") as f:
    f.write(f"Overall Best Damage: {outcomes[0]}\n")
    f.write(f"best_main_stats: {outcomes[1]}\n")
    f.write(f"best_allocation: {outcomes[2]}\n")
    f.write(f"best_total_stats: {outcomes[3]}\n")
    f.write(f"best_disk_set_option: {outcomes[4]}\n")
    f.write("\n\n")
# recommendations_path = "results/ellen_recommendation.txt"


New Best Damage:  146360.53061844513
best_main_stats: {'disk_4': 'ATK_%', 'disk_5': 'ATK_%', 'disk_6': 'ATK_%'}
best_allocation: {'Flat_ATK': 0, 'ATK_%': 0, 'CRIT_Rate': 0, 'CRIT_DMG': 10, 'Flat_PEN': 24}
best_total_stats: {'Level': 60, 'Level_Coefficient': 281, 'DEF_Reduction': 0, 'Stun_Bonus': 0, 'Total_HP': 9872.3, 'Total_DEF': 797.3, 'Total_ATK': 4155.728485, 'CRIT_Rate': 0.47400000000000003, 'CRIT_DMG': 0.98, 'PEN_Ratio': 0, 'Impact': 93, 'Anomaly_Mastery': 94, 'Anomaly_Proficiency': 93, 'Energy_Regen': 1.2, 'Buff_Level_Multiplier': 1.9970999999999999, 'DMG%_Multiplier': 1.425, 'DMG_Taken_Multiplier': 1}
best_disk_set_option: ('Hormone_Punk', 'Inferno_Metal')



New Best Damage:  146562.59086525592
best_main_stats: {'disk_4': 'ATK_%', 'disk_5': 'ATK_%', 'disk_6': 'ATK_%'}
best_allocation: {'Flat_ATK': 0, 'ATK_%': 0, 'CRIT_Rate': 0, 'CRIT_DMG': 11, 'Flat_PEN': 23}
best_total_stats: {'Level': 60, 'Level_Coefficient': 281, 'DEF_Reduction': 0, 'Stun_Bonus': 0, 'Total_HP': 9872.3, 'Tot